# LT5-LE7 Sidelap NDVI Comparison

In [19]:
import ee
ee.Initialize()

# Detroit MSA bounds
detroit_msa = ee.FeatureCollection('ft:1QZFRMLeORVfc9sYNOOKD2WyOlEyI8gmj-g3CdTlH', 'geometry')

# Water mask
water = ee.Image(1).where(ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('max_extent'), ee.Image(0))

## Functions

In [39]:
def cloud_mask(img):
    '''
    Applies a cloud mask to an image based on the QA band of the
    Landsat data.
    '''
    # Bits 3 and 5 are cloud shadow and cloud, respectively
    cloud_shadow_bit_mask = ee.Number(2).pow(3).int()
    clouds_bit_mask = ee.Number(2).pow(5).int()
    water_mask = ee.Number(2).pow(2).int() # Bit 2 is water

    # Get the pixel QA band
    qa = img.select('pixel_qa')

    # Both flags should be set to zero, indicating clear conditions
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0)\
        .And(qa.bitwiseAnd(clouds_bit_mask).eq(0))\
        .And(qa.bitwiseAnd(water_mask).eq(0))\
        .reduceNeighborhood(ee.Reducer.anyNonZero(), ee.Kernel.square(1))
    # NOTE: Also mask those pixels that are adjacent to cloud, cloud shadow, or water

    # Return the masked image, scaled to [0, 1]
    return img.updateMask(mask).divide(10000)\
        .select(
            ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'],
            ['blue', 'green', 'red', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa'])


def ndvi(img):
    '''
    Applies the NDVI transformation to a single image.
    '''
    return img.normalizedDifference(['NIR', 'red'])


def group_sidelap_images(img, prev):
    '''
    Finds sidelap images, based on their acquisition date/time, then
    groups them together; returns an LT5 image and 1-2 overlapping LE7 images.
    '''
    # Get footprint, acquisition time
    footprint_lt5 = ee.Geometry(img.get('system:footprint'))
    acq_time_lt5 = ee.Date(img.get('system:time_start'))
  
    # Get LE7 images adjacent to the LT5 date
    le7_adj_images = le7_trial\
        .filterDate(acq_time_lt5.advance(-1, 'day'), acq_time_lt5.advance(1, 'day'))\
        .filterBounds(footprint_lt5)
        # Of those, get LE7 images that intersect footprint

    le7_sidelaps = le7_adj_images.toList(2)

    # Get two adjacent (in space and time) LE7 images, or an empty List
    return ee.List(prev).add(ee.Algorithms.If(ee.Number(le7_sidelaps.size()).gte(2),
        # Return, essentially: [this_img, [adj_img1, adj_img2]]
        ee.List([img, ee.List([le7_sidelaps.get(0), le7_sidelaps.get(1)])]), 
        # Or, if not enough adjacent images: []
        ee.List([])))


def intersect_sidelap_images(group, prev):
    '''
    From among the grouped, candidate sidelap images, returns LT5-LE7 
    image pairs corresponding to the shared/ sidelap area. Also applies
    a cloud mask to the images prior to intersection.
    '''
    lt5 = ee.Image(ee.List(group).get(0))
    lt5_fp = ee.Geometry.Polygon(ee.Geometry(lt5.get('system:footprint')).coordinates())
    le7a = ee.Image(ee.List(ee.List(group).get(1)).get(0))
    le7b = ee.Image(ee.List(ee.List(group).get(1)).get(1))
    le7a_fp = ee.Geometry.Polygon(ee.Geometry(le7a.get('system:footprint')).coordinates())
    le7b_fp = ee.Geometry.Polygon(ee.Geometry(le7b.get('system:footprint')).coordinates())
  
    # Get mean of LE7 images, apply cloud mask to LT5 and LE7
    lt5_masked = cloudMask(lt5)
    le7_masked = cloudMask(le7a).add(cloudMask(le7b)).divide(ee.Image.constant(2.0))
  
    # Get the area of their sidelap
    sidelap = ee.Geometry(lt5_fp)\
        .intersection(le7a_fp)\
        .intersection(le7b_fp)

    # Test that area of intersection is > 10e6 sq. meters
    return ee.List(prev).add(ee.Algorithms.If(ee.Number(sidelap.area()).lte(10e6), 
        ee.List([]),
        ee.List([lt5_masked.clip(sidelap), le7_masked.clip(sidelap)])))


def threshold_sidelaps_through_ndvi(pair, prev):
    '''
    Computes NDVI in those areas where the coefficient of variation of NIR
    and Red bands are both below a prescribed threshold (after Ju & Masek, 2006).
    '''
    lt5 = ee.Image(ee.List(pair).get(0))
    le7 = ee.Image(ee.List(pair).get(1))

    # Compute coef. of variation in a 3x3 window (at Landsat pixel scale)
    lt5_sd = lt5.select('NIR', 'red').reduceNeighborhood(ee.Reducer.stdDev(), 
        ee.Kernel.square(45, 'meters'))
    lt5_mean = lt5.select('NIR', 'red').reduceNeighborhood(ee.Reducer.mean(), 
        ee.Kernel.square(45, 'meters'))
    le7_sd = le7.select('NIR', 'red').reduceNeighborhood(ee.Reducer.stdDev(), 
        ee.Kernel.square(45, 'meters'))
    le7_mean = le7.select('NIR', 'red').reduceNeighborhood(ee.Reducer.mean(), 
        ee.Kernel.square(45, 'meters'))

    lt5_cv = lt5_sd.divide(lt5_mean).rename(['NIR', 'red'])
    le7_cv = le7_sd.divide(le7_mean).rename(['NIR', 'red'])

    # Calculate NDVI over those pixels with a neighborhood CV < 0.02
    thresh = ee.Image.constant(0.02)
    lt5_mask = lt5_cv.select('NIR').lt(thresh).And(lt5_cv.select('red').lt(thresh))
    le7_mask = le7_cv.select('NIR').lt(thresh).And(le7_cv.select('red').lt(thresh))
    mask = lt5_mask.And(le7_mask) # Combine the masks for pixel-level consistency

    lt5_ndvi = ndvi(lt5).updateMask(mask).rename('LT5')
    le7_ndvi = ndvi(le7).updateMask(mask).rename('LE7')

    return ee.List(prev).add(ee.List([lt5_ndvi, le7_ndvi]))

## Assets

In [21]:
lt5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')\
    .filterDate('1999-01-01', '2011-12-31')\
    .filter(ee.Filter.dayOfYear(152, 244))\
    .filterBounds(detroit_msa)\
    .filterMetadata('CLOUD_COVER_LAND', 'less_than', 80)

le7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')\
    .filterDate('1999-01-01', '2011-12-31')\
    .filter(ee.Filter.dayOfYear(152, 244))\
    .filterBounds(detroit_msa)\
    .filterMetadata('CLOUD_COVER_LAND', 'less_than', 80)

## Workspace

Here, we can change the date range for the `*_trial` image collections so that we don't exceed Earth Engine's memory. We work through all years 1999-2011, when Landsats 5 and 7 were both flying, exporting the intersected sidelap areas of both LT5 and LE7 for all available images. Just based on the number of images available in each range of years, and the user memory allocated by Earth Engine at the time, I had to break this up into three groups, all inclusive year ranges:

- 1999-2005 
- 2006-2009
- 2009-2011

In [58]:
lt5_trial = lt5.filterDate('2009-01-01', '2011-12-31').map(lambda img: img.updateMask(water))
le7_trial = le7.filterDate('2009-01-01', '2011-12-31').map(lambda img: img.updateMask(water))

# Get initial sidelap pairs
lt5_le7_sidelap_groups = lt5_trial.iterate(group_sidelap_images, ee.List([]))

# Remove empty lists, then create pairs of intersected images, then 
#   remove empty lists again
lt5_le7_sidelap_groups = ee.List(lt5_le7_sidelap_groups)\
    .removeAll(ee.List([ee.List([])]))

lt5_le7_sidelap_pairs = ee.List(lt5_le7_sidelap_groups)\
    .iterate(intersect_sidelap_images, ee.List([]))

lt5_le7_sidelap_pairs = ee.List(lt5_le7_sidelap_pairs)\
    .removeAll(ee.List([ee.List([])]))

# Finally, apply NDVI to images after further masking by CV of NIR, Red bands
lt5_le7_ndvi_pairs = lt5_le7_sidelap_pairs.iterate(threshold_sidelaps_through_ndvi, ee.List([]))

# Group the LT5 and LE7 NDVI images as bands in a new Image
ndvi_series = ee.List(lt5_le7_ndvi_pairs)\
    .iterate(lambda pair, prev: ee.ImageCollection(prev).merge(ee.ImageCollection([
        ee.Image(ee.List(pair).get(0)).addBands(ee.Image(ee.List(pair).get(1)))
])), ee.ImageCollection([]))
    
for i in xrange(0, ee.ImageCollection(ndvi_series).size().getInfo()):
    img = ee.Image(ee.ImageCollection(ndvi_series).toList(1, i).get(0))
    task = ee.batch.Export.image.toDrive(image = img, 
        description = 'NDVI_2009-2011_%d' % i,
        folder = 'EarthEngine', scale = 30, crs = 'EPSG:32617')
    task.start()